In [1]:
import gymnasium as gym
import random
from IPython.display import clear_output
%config NotebookApp.iopub_msg_rate_limit=10000
import time
import pygame

In [2]:
desc=["SFFF",  #S=StartingPoint_Safe
      "FHHH",  #F=FrozenSurface_Safe
      "FFFF",
      "HFHF",
      "FFGF"  #H=Hole_NotSafe
      ]       #G=Goal

mazeSize = [len(desc),len(desc[0])]
env = gym.make('FrozenLake-v1', desc=desc, map_name="3x3", is_slippery=False, render_mode="human")

observation, info = env.reset()

In [9]:
pygame.quit()

: 

In [4]:
#Hyperparameters
epsilonValue = 0.05
alpha = 0.5
qTable = {}
currentState = 0

def resetTable():
    global qTable
    qTable = {}
    for i in range(mazeSize[0]*mazeSize[1]):
        qTable[i] = [0,0,0,0] 
    global currentState
    currentState = 0

In [5]:
def calcPossibleMoves(state):
    possibleMoves = []
    if (state+1) % mazeSize[1] != 0:
        possibleMoves.append(2)
        
    if (state+1) % mazeSize[1] != 1:
        if not state == 0:
            possibleMoves.append(0)
        
    if state > (mazeSize[1]-1):
        if not state == 0:
            possibleMoves.append(3)
    
    if state < ((mazeSize[0] * mazeSize[1]) - mazeSize[1]):
        possibleMoves.append(1)
        
    return possibleMoves

In [6]:
def nextStep(state):
    x = max(qTable[state])
    if x >= epsilonValue:
        nextMove = qTable[state].index(x)
    else:
        possMoves = calcPossibleMoves(state)
        nextMove = possMoves[random.randrange(len(possMoves))]
    return nextMove

In [7]:
convergenceThresh = 0.125
def updateTable(direction, nextState, reward):
    didConverge = False
    global currentState
    updated = qTable[currentState][direction] + alpha*(reward + max(qTable[nextState]) - qTable[currentState][direction])
    changeInQ = abs(qTable[currentState][direction] - updated)
    if changeInQ < convergenceThresh:
        if changeInQ > 0:
            didConverge = True
    qTable[currentState][direction] = updated
    currentState = nextState
    return didConverge, changeInQ

In [8]:
maxEpisodes = 1000
currentEpisode = 1
converged = False

resetTable()
env.reset()
start_time = time.time()
while currentEpisode <= maxEpisodes:
    if converged:
        break
        
    direction = nextStep(currentState)
    observation, reward, terminated, truncated, info = env.step(direction)
    
    converged, changeInQ = updateTable(direction, observation, reward)

    if terminated or truncated or converged:
        observation, info = env.reset()
        if not converged:
            currentEpisode += 1

        
    if converged:
        end_time = time.time()

    clear_output(wait=True)
    print("Episode: " + str(currentEpisode) + "/" + str(maxEpisodes))
    print("Time: " + str(round(time.time()-start_time, 3)) + " sec")
    print("Q-Table:")
    for i in range(len(qTable)):
        print(str(i) + ": " + str(qTable[i]))
    print("change in Q: " + str(changeInQ))
            
if converged:
    duration = end_time - start_time
    print(str(round(duration, 3)) + " seconds to converge")
else:
    print("No convergence")

Episode: 12/1000
Time: 14.307 sec
Q-Table:
0: [0, 0.0, 0.0, 0]
1: [0.0, 0.0, 0.0, 0]
2: [0, 0.0, 0, 0]
3: [0, 0, 0, 0]
4: [0, 0.0, 0.0, 0.0]
5: [0.0, 0.0, 0.0, 0.0]
6: [0, 0.0, 0, 0]
7: [0, 0, 0, 0]
8: [0, 0, 0.0, 0.0]
9: [0.0, 0.0, 0.0, 0.0]
10: [0.0, 0, 0, 0]
11: [0, 0, 0, 0]
12: [0, 0, 0, 0]
13: [0, 0, 0, 0.0]
14: [0, 0, 0, 0]
15: [0, 0, 0, 0]
16: [0, 0, 0, 0]
17: [0, 0, 0, 0]
18: [0, 0, 0, 0]
19: [0, 0, 0, 0]
change in Q: 0.0


KeyboardInterrupt: 

In [ ]:
env.close()

: 